In [21]:
import numpy as np
from scipy.linalg import block_diag
from cvxopt import matrix, solvers ,glpk
solvers.options['show_progress'] = False
#np.random.seed(7000)

In [25]:
#SYSTEM1
def _system1(sparse_,down_,right_):
    row_constrinats=block_diag(*sparse_)
    column_constraints=np.concatenate(np.apply_along_axis(lambda x: block_diag(*x),1,sparse_)).T
    A=np.append(row_constrinats,column_constraints,axis=0)
    A=A.astype("d")
    rows=np.where(np.sum(A,axis=1)!=0)[0]
    cols=np.where(np.sum(A,axis=0)!=0)[0]
    A=A[:,cols][rows,:]
    variables=A.shape[1]
    b=np.concatenate([right_, down_], axis=0) 
    b=b.astype("d")
    c=np.ones(variables)
    rank=np.linalg.matrix_rank(A)
    A=A[:rank,:]
    b=b[:rank]
    g=matrix(-1*np.identity(variables,dtype='d'))
    H=matrix(np.zeros(variables,dtype='d'))
    #print(A,b,g,H)
    sol=solvers.lp(matrix(-1*c),G=g,h=H,A=matrix(A),b=matrix(b))
    return sol

def make_infeasible(sparse,_right,_down):
    init=_right[0]
    _right[0]= sum(sparse[0,:]*down)+1
    i=1
    while sum(_right)!=sum(_down):
        new=int(_right[i]/6)
        old=_right[i]
        alloc=min(_right[0]-init,old-new)
        _right[i]=old-alloc
        init=init+alloc
        i=i+1
    return _right

In [26]:
#system2
def _system2(sparse,down,right):
    A_= sparse
    b_=np.apply_along_axis(lambda x: np.sum(x*down),1,sparse)-right
    b_[-1]=right[-1]
    var_=A_.shape[1]
    A=A_[-1,:]
    b=b_[-1]
    A_[:-1,:]
    b_[:-1]
    A_=np.concatenate([A_,-1*np.identity(var_),np.identity(var_)])
    b_=np.concatenate([b_,np.zeros(var_),down])
    c_= np.zeros(var_)
    sol_=solvers.lp(matrix(-1*c_),G=matrix(A_.astype(np.double)),h=matrix(b_.astype(np.double)),A=matrix(A.astype(np.double)).T,b=matrix(b.astype(np.double)))
    return sol_

In [27]:
is_solved=True
i=0
while is_solved:
    m=np.random.randint(5,10)
    n=np.random.randint(5,10)
    #create sparse matrix
    mat=np.random.randint(15,40,(m,n))
    p=mat[:-1,:]
    p[p%3 -1 !=0]=0
    
    #remove void rows and columns
    rs=np.sum(p,axis=1)
    cs=np.sum(p,axis=0)
    rows=np.where(rs!=0)[0]
    rows=np.append(rows,m-1)
    cols=np.where(cs!=0)[0]
    mat=mat[:,cols][rows,:]

    if mat.shape[0]<=4 or mat.shape[1]<4:
        continue
    #create supply demand
    right=mat.sum(axis=1)
    down=mat.sum(axis=0)
    sparse=np.copy(mat)
    sparse[sparse>0]=1
    
    if np.random.randint(0,2)==1:
        right=make_infeasible(sparse,right,down)
        
    full_rank=np.unique(sparse,axis=0,return_index=True)
    sparse=full_rank[0]
    right=right[full_rank[1]]
    right[-1]=down.sum()-right[:-1].sum()
        
    #get sol using linear programming
    sol1=_system1(sparse,down,right)
    #solve using swapnil reduction
    sol2=_system2(sparse,down,right)
    _sparse=sparse[:-1,:]
    _right=right[:-1]
    if sol2["status"]=="optimal":
        print(np.array(sol2["x"].T))
        _down=(down-np.array(sol2["x"].T))[0]
        #remove the solution of last row after reduction and solve again using linear programming
        _sparse=_sparse[np.where(_right!=0)[0],:][:,np.where(_down!=0)[0]]
        _right=_right[np.where(_right!=0)[0]]
        _down=_down[np.where(_down!=0)[0]]
        sol3=_system1(_sparse,_down,_right)

    is_solved=False
    if i%1000==0:
        print(i)
    i=i+1
    if sol1["status"]=="optimal":
        print("+vetest")
        is_solved = (sol2["status"]=="optimal") #and (sol3["status"]=="optimal")
    else:
        print("-vetest")
        is_solved = sol2["status"]!="optimal"
        

0
-vetest
-vetest
-vetest
+vetest


In [28]:
sparse

array([[0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1, 1, 1],
       [0, 0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 1],
       [1, 0, 0, 1, 1, 0, 0],
       [1, 1, 1, 1, 1, 1, 1]])

In [29]:
print(str(sparse).replace("]","").replace("[",""))
print(down)
print(right)
print(_down)
print(_right)
print(str(_sparse).replace("]","").replace("[",""))

0 0 0 0 0 1 0
 0 0 0 0 1 1 1
 0 0 1 0 0 0 0
 0 1 0 0 0 0 1
 1 0 0 1 1 0 0
 1 1 1 1 1 1 1
[34 55 37 51 69 94 74]
[ 31  66  16  38  60 203]
[ 109.41236928   38.08100239   79.38029123   84.11694326  105.00939384]
[31 66 16 38 60]
0 0 0 0 0 1 0
 0 0 0 0 1 1 1
 0 0 1 0 0 0 0
 0 1 0 0 0 0 1
 1 0 0 1 1 0 0


In [83]:
sol2[0]=="optimal"

True

In [66]:
_system1(_sparse,_down,_right)

[[1. 1. 0. 0. 0. 0.]
 [0. 0. 1. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [1. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 1. 1.]] [ 50.  32.  50.  87.  62. 116.] [-1.00e+00 -0.00e+00 -0.00e+00 -0.00e+00 -0.00e+00 -0.00e+00]
[-0.00e+00 -1.00e+00 -0.00e+00 -0.00e+00 -0.00e+00 -0.00e+00]
[-0.00e+00 -0.00e+00 -1.00e+00 -0.00e+00 -0.00e+00 -0.00e+00]
[-0.00e+00 -0.00e+00 -0.00e+00 -1.00e+00 -0.00e+00 -0.00e+00]
[-0.00e+00 -0.00e+00 -0.00e+00 -0.00e+00 -1.00e+00 -0.00e+00]
[-0.00e+00 -0.00e+00 -0.00e+00 -0.00e+00 -0.00e+00 -1.00e+00]
 [ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]



{'dual infeasibility': None,
 'dual objective': 1.0,
 'dual slack': 8.309359201454974e-05,
 'gap': None,
 'iterations': 5,
 'primal infeasibility': None,
 'primal objective': None,
 'primal slack': None,
 'relative gap': None,
 'residual as dual infeasibility certificate': None,
 'residual as primal infeasibility certificate': 2.8359488201607e-09,
 's': None,
 'status': 'primal infeasible',
 'x': None,
 'y': <6x1 matrix, tc='d'>,
 'z': <6x1 matrix, tc='d'>}

In [85]:
np.array(sol2[1].T)

array([[64., 32., 31., -0., -0., -0., -0., 75.]])